In [1]:
# this cell combines two json files into one dataframe after all the preprocessing steps

# fucntion for date conversion of openweather data
def convert_date(ts):
    import pandas as pd
    ts=pd.to_datetime(ts, unit='s')
    year=ts.year
    month=ts.month
    day=ts.day
    hour=ts.hour
    return [year, month, day, hour]

# combining the two json files into one dataframe for a particular zone
def combine_data():
        import pandas as pd
        import json

        # reading openweather data
        with open(f"openweather.json") as f:
                openweather = json.load(f)

        # getting longitude and latitude
        longitude = openweather['coord']['lon']
        latitude = openweather['coord']['lat']

        # converting openweather data to dataframe
        arr=[convert_date(openweather['list'][i]['dt'])+[longitude,latitude,openweather['list'][0]['main']['aqi']]+
                list(openweather['list'][i]['components'].values()) for i in range(len(openweather['list']))]
        openweather = pd.DataFrame(arr, columns=['year','month','day','hour',
                'longitude','latitude','aqi','co','no','no2','o3','so2','pm2_5','pm10','nh3'])


        # reading meteo data
        with open(f"meteo.json") as f:
                meteo = json.load(f)
        
        arr=[convert_date(pd.to_datetime(meteo['hourly']['time'][i]))+[longitude,latitude,
                meteo['hourly']['temperature_2m'][i], meteo['hourly']['dewpoint_2m'][i]] for i in range(len(meteo['hourly']['time']))]
        meteo = pd.DataFrame(arr, columns=['year','month','day','hour','longitude','latitude','temperature','dewpt'])
        
        # combining the two dataframes
        pd.merge(openweather, meteo, on=['year','month','day','hour','longitude','latitude']).drop([8734]).sort_index(kind='merge').to_csv('combined_data.csv', index=False)

In [2]:
# function to get weather data and store it in csv file

def getweather(lon, lat):
    # openweathermap api
    # capturing the api response

    import requests
    # url = f"https://api.openweathermap.org/data/2.5/air_pollution/history?lat={coord[zone][1][0]}&lon={coord[zone][1][1]}&start=1609459200&end=1640995200&appid=8d9efe95caf3922348a458d615abeb8c"
    url = f"https://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start=1609459200&end=1667170800&appid=8d9efe95caf3922348a458d615abeb8c"
    url1 = f"https://archive-api.open-meteo.com/v1/era5?latitude={lat}&longitude={lon}&start_date=2021-01-01&end_date=2022-09-30&hourly=temperature_2m,dewpoint_2m"
    response = requests.get(url)
    response1 = requests.get(url1)
    data = response.json()
    data1 = response1.json()

    # storing the api response in a json file
    with open(f"openweather.json","w") as f:
        import json
        json.dump(data,f)

    # storing the api response in a json file
    with open(f"meteo.json","w") as f:
        import json
        json.dump(data1,f)
    combine_data()

In [3]:
def plot():
    import pandas as pd

    df=pd.read_csv('combined_data.csv')
    df.head()

    # converting the date column to datetime
    date_time = pd.to_datetime(df[['year','month','day','hour']], format='%Y-%m-%d %H:%M:%S')
    # df.drop(['month','day','hour','zone','longitude','latitude','aqi'], axis=1, inplace=True)
    df['Date Time'] = date_time

    # reanaming year to date
    # df.rename(columns={'year':'Date Time'}, inplace=True)
    # return df

    # creating a plotly line chart
    import plotly.express as px
    return px.line(df, x='Date Time', y='pm10', title='PM10')

In [4]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import dash
from geopy.geocoders import Nominatim
import time
import plotly.express as px
import pandas as pd

app = Dash(__name__)

# app.layout = html.Div([
#     html.Label('Enter a city to see its weather'),
#     dcc.Input(id='num', type='text', debounce=True, style={'width': '100%'}, autoFocus=True),
#     html.P(id='out')
# ])

# creating app layout for taking input from user and plotting the graph
app.layout = html.Div([
    html.Div([
        html.Label('Enter a city to see its weather'),
        dcc.Input(id='num', type='text', debounce=True, style={'width': '100%'}, autoFocus=True),
        # html.P(id='out'),
        dcc.Graph(id='out')
    ])
])

@app.callback(
    Output('out', 'children'),
    Input('num', 'value')
)
def show(num):
    if num is None:
        # PreventUpdate prevents ALL outputs updating
        raise dash.exceptions.PreventUpdate

    # converting input city to coordinates
    # calculating execution time
    start = time.time()
    geolocator = Nominatim(user_agent='myapplication')
    location = geolocator.geocode("{}".format(num))
    getweather(location.longitude, location.latitude)
    end = time.time()
    print(end - start)
    return plot(),' '

if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [25/Nov/2022 23:31:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2022 23:31:27] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2022 23:31:27] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2022 23:31:27] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Nov/2022 23:31:27] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Nov/2022 23:31:27] "POST /_dash-update-component HTTP/1.1" 204 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/dash/dash.py", line 1274, in dispatch
    ctx.run(
  File "/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/dash/_callback.py", line 440, in add_context
    output_value = func(*func_args, **

127.0.0.1 - - [25/Nov/2022 23:31:29] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [25/Nov/2022 23:31:40] "GET /_reload-hash HTTP/1.1" 200 -


8.431909084320068


127.0.0.1 - - [25/Nov/2022 23:31:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2022 23:31:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2022 23:31:54] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2022 23:31:54] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2022 23:31:54] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Nov/2022 23:31:54] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [25/Nov/2022 23:31:54] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Nov/2022 23:31:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2022 23:31:56] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2022 23:31:56] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2022 23:31:56] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Nov/2022 23:31:56] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js 

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/dash/dash.py", line 1274, in dispatch
    ctx.run(
  File "/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/dash/_callback.py", line 440, in add_context
    output_value = func(*func_args, **

127.0.0.1 - - [25/Nov/2022 23:32:00] "POST /_dash-update-component HTTP/1.1" 500 -
